In [6]:
from __future__ import print_function, division
import tensorflow as tf
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K
import os, math, cv2, glob, random, time
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import matplotlib.pyplot as plt


import numpy as np

class SGAN:
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(
            loss=['binary_crossentropy', 'categorical_crossentropy'],
            loss_weights=[0.5, 0.5],
            optimizer=optimizer,
            metrics=['accuracy']
        )

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        noise = Input(shape=(100,))
        img = self.generator(noise)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid, _ = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(noise, valid)
        self.combined.compile(loss=['binary_crossentropy'], optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(1, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())

        model.summary()

        img = Input(shape=self.img_shape)

        features = model(img)
        valid = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes+1, activation="softmax")(features)

        return Model(img, [valid, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Class weights:
        # To balance the difference in occurences of digit class labels.
        # 50% of labels that the discriminator trains on are 'fake'.
        # Weight = 1 / frequency
        half_batch = batch_size // 2
        cw1 = {0: 1, 1: 1}
        cw2 = {i: self.num_classes / half_batch for i in range(self.num_classes)}
        cw2[self.num_classes] = 1 / half_batch

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # One-hot encoding of labels
            labels = to_categorical(y_train[idx], num_classes=self.num_classes+1)
            fake_labels = to_categorical(np.full((batch_size, 1), self.num_classes), num_classes=self.num_classes+1)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, [valid, labels], class_weight=[cw1, cw2])
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, fake_labels], class_weight=[cw1, cw2])
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            g_loss = self.combined.train_on_batch(noise, valid, class_weight=[cw1, cw2])

            # Plot the progress
            print ("%d [D loss: %f, acc: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[3], 100*d_loss[4], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()

    def save_model(self):

        def save(model, model_name):
            model_path = "saved_model/%s.json" % model_name
            weights_path = "saved_model/%s_weights.hdf5" % model_name
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])

        save(self.generator, "mnist_sgan_generator")
        save(self.discriminator, "mnist_sgan_discriminator")
        save(self.combined, "mnist_sgan_adversarial")


if __name__ == '__main__':
    with tf.device('/device:gpu:0'):
        sgan = SGAN()
        sgan.train(epochs=20000, batch_size=32, sample_interval=50)



Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 8, 8, 64)         

/home/zahid/anaconda3/envs/work_env/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.911640, acc: 45.31%, op_acc: 9.38%] [G loss: 0.600083]


/home/zahid/anaconda3/envs/work_env/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.909771, acc: 54.69%, op_acc: 3.12%] [G loss: 0.700716]
2 [D loss: 0.764914, acc: 53.12%, op_acc: 9.38%] [G loss: 0.803856]
3 [D loss: 0.734592, acc: 65.62%, op_acc: 4.69%] [G loss: 1.134770]
4 [D loss: 0.716798, acc: 76.56%, op_acc: 6.25%] [G loss: 1.050511]
5 [D loss: 0.674504, acc: 81.25%, op_acc: 9.38%] [G loss: 1.019759]
6 [D loss: 0.623968, acc: 93.75%, op_acc: 4.69%] [G loss: 1.184072]
7 [D loss: 0.576506, acc: 89.06%, op_acc: 14.06%] [G loss: 0.881663]
8 [D loss: 0.572423, acc: 89.06%, op_acc: 12.50%] [G loss: 0.681874]
9 [D loss: 0.504079, acc: 92.19%, op_acc: 20.31%] [G loss: 0.486258]
10 [D loss: 0.555403, acc: 90.62%, op_acc: 23.44%] [G loss: 0.417777]
11 [D loss: 0.480433, acc: 95.31%, op_acc: 39.06%] [G loss: 0.334377]
12 [D loss: 0.514132, acc: 98.44%, op_acc: 31.25%] [G loss: 0.361676]
13 [D loss: 0.394653, acc: 96.88%, op_acc: 46.88%] [G loss: 0.225983]
14 [D loss: 0.428345, acc: 98.44%, op_acc: 51.56%] [G loss: 0.113489]
15 [D loss: 0.406061, acc: 95.31%, 

...

19525 [D loss: 0.324286, acc: 71.88%, op_acc: 59.38%] [G loss: 1.260338]
19526 [D loss: 0.324257, acc: 71.88%, op_acc: 75.00%] [G loss: 1.077379]
19527 [D loss: 0.348799, acc: 68.75%, op_acc: 59.38%] [G loss: 1.113447]
19528 [D loss: 0.389397, acc: 65.62%, op_acc: 54.69%] [G loss: 1.256730]
19529 [D loss: 0.335070, acc: 71.88%, op_acc: 60.94%] [G loss: 1.072539]
19530 [D loss: 0.310585, acc: 75.00%, op_acc: 60.94%] [G loss: 1.524766]
19531 [D loss: 0.333555, acc: 71.88%, op_acc: 57.81%] [G loss: 0.959796]
19532 [D loss: 0.446850, acc: 59.38%, op_acc: 54.69%] [G loss: 1.478792]
19533 [D loss: 0.463040, acc: 43.75%, op_acc: 56.25%] [G loss: 1.221559]
19534 [D loss: 0.326011, acc: 81.25%, op_acc: 59.38%] [G loss: 1.038667]
19535 [D loss: 0.391736, acc: 54.69%, op_acc: 60.94%] [G loss: 1.431948]
19536 [D loss: 0.333324, acc: 70.31%, op_acc: 56.25%] [G loss: 1.070458]
19537 [D loss: 0.272264, acc: 81.25%, op_acc: 62.50%] [G loss: 1.358132]
19538 [D loss: 0.314765, acc: 73.44%, op_acc: 62.50

19642 [D loss: 0.422382, acc: 56.25%, op_acc: 60.94%] [G loss: 0.510988]
19643 [D loss: 0.410008, acc: 56.25%, op_acc: 56.25%] [G loss: 1.256528]
19644 [D loss: 0.389114, acc: 53.12%, op_acc: 56.25%] [G loss: 0.834743]
19645 [D loss: 0.329631, acc: 81.25%, op_acc: 62.50%] [G loss: 0.958555]
19646 [D loss: 0.381640, acc: 50.00%, op_acc: 60.94%] [G loss: 1.209978]
19647 [D loss: 0.444226, acc: 56.25%, op_acc: 51.56%] [G loss: 1.325149]
19648 [D loss: 0.336259, acc: 71.88%, op_acc: 57.81%] [G loss: 1.440143]
19649 [D loss: 0.397988, acc: 56.25%, op_acc: 57.81%] [G loss: 1.718668]
19650 [D loss: 0.426389, acc: 51.56%, op_acc: 59.38%] [G loss: 2.014482]
19651 [D loss: 0.349929, acc: 70.31%, op_acc: 56.25%] [G loss: 1.518812]
19652 [D loss: 0.425819, acc: 71.88%, op_acc: 56.25%] [G loss: 1.351718]
19653 [D loss: 0.424417, acc: 48.44%, op_acc: 64.06%] [G loss: 1.404332]
19654 [D loss: 0.457461, acc: 50.00%, op_acc: 60.94%] [G loss: 1.210733]
19655 [D loss: 0.511949, acc: 40.62%, op_acc: 50.00

19757 [D loss: 0.395595, acc: 56.25%, op_acc: 51.56%] [G loss: 0.708394]
19758 [D loss: 0.370520, acc: 64.06%, op_acc: 60.94%] [G loss: 1.178660]
19759 [D loss: 0.340127, acc: 68.75%, op_acc: 59.38%] [G loss: 1.064102]
19760 [D loss: 0.290781, acc: 82.81%, op_acc: 62.50%] [G loss: 1.300546]
19761 [D loss: 0.481164, acc: 53.12%, op_acc: 54.69%] [G loss: 0.845512]
19762 [D loss: 0.354337, acc: 70.31%, op_acc: 60.94%] [G loss: 0.822059]
19763 [D loss: 0.451105, acc: 53.12%, op_acc: 57.81%] [G loss: 1.113593]
19764 [D loss: 0.411010, acc: 57.81%, op_acc: 54.69%] [G loss: 1.066207]
19765 [D loss: 0.421322, acc: 54.69%, op_acc: 54.69%] [G loss: 0.991974]
19766 [D loss: 0.366011, acc: 71.88%, op_acc: 56.25%] [G loss: 0.920226]
19767 [D loss: 0.433922, acc: 59.38%, op_acc: 51.56%] [G loss: 0.950381]
19768 [D loss: 0.417056, acc: 56.25%, op_acc: 51.56%] [G loss: 1.072173]
19769 [D loss: 0.253803, acc: 79.69%, op_acc: 57.81%] [G loss: 1.155783]
19770 [D loss: 0.446724, acc: 53.12%, op_acc: 60.94

19874 [D loss: 0.347657, acc: 73.44%, op_acc: 59.38%] [G loss: 1.213826]
19875 [D loss: 0.456532, acc: 60.94%, op_acc: 53.12%] [G loss: 1.153669]
19876 [D loss: 0.386968, acc: 51.56%, op_acc: 54.69%] [G loss: 1.353246]
19877 [D loss: 0.308963, acc: 70.31%, op_acc: 64.06%] [G loss: 1.545831]
19878 [D loss: 0.353051, acc: 67.19%, op_acc: 59.38%] [G loss: 1.187353]
19879 [D loss: 0.338139, acc: 70.31%, op_acc: 53.12%] [G loss: 1.282480]
19880 [D loss: 0.265948, acc: 81.25%, op_acc: 60.94%] [G loss: 1.143515]
19881 [D loss: 0.412826, acc: 62.50%, op_acc: 53.12%] [G loss: 1.801497]
19882 [D loss: 0.376207, acc: 67.19%, op_acc: 62.50%] [G loss: 1.059319]
19883 [D loss: 0.504114, acc: 45.31%, op_acc: 54.69%] [G loss: 0.927756]
19884 [D loss: 0.427933, acc: 64.06%, op_acc: 54.69%] [G loss: 1.055024]
19885 [D loss: 0.280105, acc: 81.25%, op_acc: 60.94%] [G loss: 1.370234]
19886 [D loss: 0.226807, acc: 93.75%, op_acc: 59.38%] [G loss: 1.580024]
19887 [D loss: 0.378148, acc: 64.06%, op_acc: 60.94

19991 [D loss: 0.454974, acc: 54.69%, op_acc: 50.00%] [G loss: 1.377071]
19992 [D loss: 0.512973, acc: 40.62%, op_acc: 54.69%] [G loss: 1.147503]
19993 [D loss: 0.357795, acc: 68.75%, op_acc: 64.06%] [G loss: 1.325233]
19994 [D loss: 0.469827, acc: 45.31%, op_acc: 57.81%] [G loss: 1.267091]
19995 [D loss: 0.240183, acc: 90.62%, op_acc: 60.94%] [G loss: 1.540216]
19996 [D loss: 0.404346, acc: 54.69%, op_acc: 57.81%] [G loss: 2.212359]
19997 [D loss: 0.467716, acc: 54.69%, op_acc: 53.12%] [G loss: 0.960473]
19998 [D loss: 0.465360, acc: 57.81%, op_acc: 51.56%] [G loss: 0.852443]
19999 [D loss: 0.358711, acc: 60.94%, op_acc: 54.69%] [G loss: 1.040708]


In [17]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


1 Physical GPUs, 1 Logical GPU


RuntimeError: Device placement logging must be set at program startup

In [34]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
with tf.device('/device:gpu:0'):
    model.fit(x_train, y_train, epochs=5)
print(model.evaluate(x_test, y_test))

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 48us/sample - loss: 0.2213 - accuracy: 0.9357
Epoch 2/5
60000/60000 [==============================] - 3s 45us/sample - loss: 0.0980 - accuracy: 0.9699
Epoch 3/5
60000/60000 [==============================] - 3s 45us/sample - loss: 0.0686 - accuracy: 0.9787
Epoch 4/5
60000/60000 [==============================] - 3s 43us/sample - loss: 0.0540 - accuracy: 0.9827
Epoch 5/5
10000/10000 [==============================] - 0s 35us/sample - loss: 0.0673 - accuracy: 0.9814
[0.06728456982622157, 0.9814]


In [39]:
tf.keras.backend.clear_session()